# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Dec 15 2022  8:19AM  253153        17      8902943   
1  Dec 15 2022  7:56AM  253152        19      8905527   
2  Dec 15 2022  7:40AM  253149        19  ADV80009135   
3  Dec 15 2022  7:40AM  253149        19  ADV80009136   
4  Dec 15 2022  7:40AM  253149        19  ADV80009137   
5  Dec 15 2022  7:40AM  253149        19  ADV80009138   
6  Dec 15 2022  7:40AM  253149        19  ADV80009139   
7  Dec 15 2022  7:40AM  253149        19  ADV80009144   
8  Dec 15 2022  7:40AM  253149        19  ADV80009145   
9  Dec 15 2022  7:40AM  253149        19  ADV80009147   

                          Customer ShipmentStatus  
0  Brookfield Pharmaceuticals, LLC       Released  
1                   Graystone, LLC       Released  
2              AdvaGen Pharma, Ltd       Released  
3              AdvaGen Pharma, Ltd       Released  
4              AdvaGen Pharma, Ltd       Released  
5              AdvaGen Pharma, Ltd       Released  
6              AdvaGen Pharma, Ltd       Released  
7              AdvaGen Pharma, Ltd       Released  
8              AdvaGen Pharma, Ltd       Released  
9              AdvaGen Pharma, Ltd       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
26  253142       Released          1
27  253146      Executing          1
28  253149       Released         31
29  253152       Released          1
30  253153       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
253142                NaN        NaN       1.0
253146                NaN        1.0       NaN
253149                NaN        NaN      31.0
253152                NaN        NaN       1.0
253153                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
253064                0.0        0.0      22.0
253083                0.0        0.0      13.0
253087               10.0        0.0      15.0
253089                0.0        0.0       1.0
253090                0.0        0.0      43.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
253064                  0          0        22
253083                  0          0        13
253087                 10          0        15
253089                  0          0         1
253090                  0          0        43

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               253064          0          0        22
1               253083          0          0        13
2               253087         10          0        15
3               253089          0          0         1
4               253090          0          0        43

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               253064                           22
1               253083                           13
2               253087        10                 15
3               253089                            1
4               253090                           43

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Dec 15 2022  8:19AM  253153        17  Brookfield Pharmaceuticals, LLC
1   Dec 15 2022  7:56AM  253152        19                   Graystone, LLC
2   Dec 15 2022  7:40AM  253149        19              AdvaGen Pharma, Ltd
33  Dec 15 2022  7:26AM  253146        22                NBTY Global, Inc.
34  Dec 14 2022  4:48PM  253142        10                        SugarBear
35  Dec 14 2022  3:59PM  253139        20   Granules Pharmaceuticals, Inc.
36  Dec 14 2022  3:37PM  253138        19           GCH Granules USA, Inc.
37  Dec 14 2022  3:35PM  253137        19              AdvaGen Pharma, Ltd
44  Dec 14 2022  3:32PM  253136        19             Emersa Waterbox, LLC
45  Dec 14 2022  3:23PM  253135        20                   Else Nutrition

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Dec 15 2022  8:19AM  253153        17  Brookfield Pharmaceuticals, LLC   
1  Dec 15 2022  7:56AM  253152        19                   Graystone, LLC   
2  Dec 15 2022  7:40AM  253149        19              AdvaGen Pharma, Ltd   
3  Dec 15 2022  7:26AM  253146        22                NBTY Global, Inc.   
4  Dec 14 2022  4:48PM  253142        10                        SugarBear   
5  Dec 14 2022  3:59PM  253139        20   Granules Pharmaceuticals, Inc.   
6  Dec 14 2022  3:37PM  253138        19           GCH Granules USA, Inc.   
7  Dec 14 2022  3:35PM  253137        19              AdvaGen Pharma, Ltd   
8  Dec 14 2022  3:32PM  253136        19             Emersa Waterbox, LLC   
9  Dec 14 2022  3:23PM  253135        20                   Else Nutrition   

  Completed Executing Released  
0                            1  
1                            1  
2                           31  
3                   1           
4                            1  
5                            1  
6                            1  
7                            7  
8                            1  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Dec 15 2022  8:19AM  253153        17  Brookfield Pharmaceuticals, LLC   
1  Dec 15 2022  7:56AM  253152        19                   Graystone, LLC   
2  Dec 15 2022  7:40AM  253149        19              AdvaGen Pharma, Ltd   
3  Dec 15 2022  7:26AM  253146        22                NBTY Global, Inc.   
4  Dec 14 2022  4:48PM  253142        10                        SugarBear   
5  Dec 14 2022  3:59PM  253139        20   Granules Pharmaceuticals, Inc.   
6  Dec 14 2022  3:37PM  253138        19           GCH Granules USA, Inc.   
7  Dec 14 2022  3:35PM  253137        19              AdvaGen Pharma, Ltd   
8  Dec 14 2022  3:32PM  253136        19             Emersa Waterbox, LLC   
9  Dec 14 2022  3:23PM  253135        20                   Else Nutrition   

  Released Executing Completed  
0        1                      
1        1                      
2       31                      
3                  1            
4        1                      
5        1                      
6        1                      
7        7                      
8        1                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Dec 15 2022  8:19AM  253153        17  Brookfield Pharmaceuticals, LLC   
1  Dec 15 2022  7:56AM  253152        19                   Graystone, LLC   
2  Dec 15 2022  7:40AM  253149        19              AdvaGen Pharma, Ltd   
3  Dec 15 2022  7:26AM  253146        22                NBTY Global, Inc.   
4  Dec 14 2022  4:48PM  253142        10                        SugarBear   

  Released Executing Completed  
0        1                      
1        1                      
2       31                      
3                  1            
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Dec 15 2022  8:19AM  253153        17  Brookfield Pharmaceuticals, LLC   
1  Dec 15 2022  7:56AM  253152        19                   Graystone, LLC   
2  Dec 15 2022  7:40AM  253149        19              AdvaGen Pharma, Ltd   
3  Dec 15 2022  7:26AM  253146        22                NBTY Global, Inc.   
4  Dec 14 2022  4:48PM  253142        10                        SugarBear   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       1.0        NaN        NaN  
2      31.0        NaN        NaN  
3       NaN        1.0        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Dec 15 2022  8:19AM  253153        17  Brookfield Pharmaceuticals, LLC   
1  Dec 15 2022  7:56AM  253152        19                   Graystone, LLC   
2  Dec 15 2022  7:40AM  253149        19              AdvaGen Pharma, Ltd   
3  Dec 15 2022  7:26AM  253146        22                NBTY Global, Inc.   
4  Dec 14 2022  4:48PM  253142        10                        SugarBear   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       1.0        0.0        0.0  
2      31.0        0.0        0.0  
3       0.0        1.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2277926      63.0        0.0       17.0
15         1012405      47.0        0.0        0.0
16          253125       1.0        0.0        0.0
17          253153       1.0        0.0        0.0
19         2025085      62.0        0.0        0.0
20         1012479       3.0        1.0        0.0
21          253132       1.0        0.0        0.0
22          253146       0.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2277926      63.0        0.0       17.0
1        15  1012405      47.0        0.0        0.0
2        16   253125       1.0        0.0        0.0
3        17   253153       1.0        0.0        0.0
4        19  2025085      62.0        0.0        0.0
5        20  1012479       3.0        1.0        0.0
6        21   253132       1.0        0.0        0.0
7        22   253146       0.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      63.0        0.0       17.0
1        15      47.0        0.0        0.0
2        16       1.0        0.0        0.0
3        17       1.0        0.0        0.0
4        19      62.0        0.0        0.0
5        20       3.0        1.0        0.0
6        21       1.0        0.0        0.0
7        22       0.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   63.0
1        15  Released   47.0
2        16  Released    1.0
3        17  Released    1.0
4        19  Released   62.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16   17    19   20   21   22
Status                                              
Completed  17.0   0.0  0.0  0.0   0.0  0.0  0.0  0.0
Executing   0.0   0.0  0.0  0.0   0.0  1.0  0.0  1.0
Released   63.0  47.0  1.0  1.0  62.0  3.0  1.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16   17    19   20   21   22
0          Completed  17.0   0.0  0.0  0.0   0.0  0.0  0.0  0.0
1          Executing   0.0   0.0  0.0  0.0   0.0  1.0  0.0  1.0
2           Released  63.0  47.0  1.0  1.0  62.0  3.0  1.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16   17    19   20   21   22
0  Completed  17.0   0.0  0.0  0.0   0.0  0.0  0.0  0.0
1  Executing   0.0   0.0  0.0  0.0   0.0  1.0  0.0  1.0
2   Released  63.0  47.0  1.0  1.0  62.0  3.0  1.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()